In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import logging
import sklearn.metrics
import json

2024-07-04 19:05:00.956786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 19:05:00.956886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 19:05:01.095545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
ds = pd.read_csv('/kaggle/input/leash-bio-belka-chunk/test_normalized/test_normalized.csv')

In [3]:
data_cols = [
 'b1LogP',
 'b1MolMR',
 'b1TPSA',
 'b1FractionCSP3',
 'b1NumHeteroatoms',
 'b1MolecularWeight',
 'b1ExactMass',
 'b1NumRotatableBonds',
 'b1NumValenceElectrons',
 'b1BondCount',
 'b1NumHAcceptors',

 'b2LogP',
 'b2MolMR',
 'b2TPSA',
 'b2FractionCSP3',
 'b2NumValenceElectrons',
 'b2MolecularWeight',
 'b2ExactMass',
 'b2NumRotatableBonds',
 'b2BondCount',
 'b2NumHeteroatoms',
 'b2NumHAcceptors',
 
 'b3LogP',
 'b3MolMR',
 'b3TPSA',
 'b3FractionCSP3',
 'b3NumHAcceptors',
 'b3MolecularWeight',
 'b3ExactMass',
 'b3BondCount',
 'b3NumValenceElectrons',
 'b3NumRotatableBonds',
 'b3NumHeteroatoms',
 'b3NumAromaticBonds',
]

In [4]:
smile_vector = tf.keras.layers.TextVectorization(max_tokens=None, split='character', standardize=None, output_sequence_length=70)
smile_vector.set_vocabulary(["", "[UNK]", "c", "C", "1", ")", "(", "O", "2", "N", "=", "n", "-", "l", "]", "[", "@", "H", "F", ".", "3", "s", "B", "r", "S", "#", "+", "o", "I", "4", "/", "5", "i"])

In [5]:
sEHModel = tf.keras.models.load_model('/kaggle/input/leash_bio_seh/tensorflow2/train-w-smile-vectors/1/sEH.keras')
sEHs = ds[ds['protein_name'] == 'sEH']
buildingblock1_smiles_vec = smile_vector(sEHs['buildingblock1_smiles'].values)
buildingblock2_smiles_vec = smile_vector(sEHs['buildingblock2_smiles'].values)
buildingblock3_smiles_vec = smile_vector(sEHs['buildingblock3_smiles'].values)
sEHpredictions = sEHModel.predict([buildingblock1_smiles_vec, buildingblock2_smiles_vec, buildingblock3_smiles_vec, sEHs[data_cols]])
sEHs['binds'] = sEHpredictions.flatten()

   78/17442 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step

I0000 00:00:1720119983.424818      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


17442/17442 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step


/tmp/ipykernel_24/2721665985.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sEHs['binds'] = sEHpredictions.flatten()


In [6]:
HSAModel = tf.keras.models.load_model('/kaggle/input/belka_bio_hsa/tensorflow2/train-w-smile-vectors/1/HSA.keras')
HSAs = ds[ds['protein_name'] == 'HSA']
buildingblock1_smiles_vec = smile_vector(HSAs['buildingblock1_smiles'].values)
buildingblock2_smiles_vec = smile_vector(HSAs['buildingblock2_smiles'].values)
buildingblock3_smiles_vec = smile_vector(HSAs['buildingblock3_smiles'].values)
HSApredictions = HSAModel.predict([buildingblock1_smiles_vec, buildingblock2_smiles_vec, buildingblock3_smiles_vec, HSAs[data_cols]])
HSAs['binds'] = HSApredictions.flatten()

17435/17435 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step


/tmp/ipykernel_24/4219568825.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HSAs['binds'] = HSApredictions.flatten()


In [7]:
BRD4Model = tf.keras.models.load_model('/kaggle/input/belka_bio_brd4/tensorflow2/train-w-smile-vectors/1/BRD4.keras')
BRD4s = ds[ds['protein_name'] == 'BRD4']
buildingblock1_smiles_vec = smile_vector(BRD4s['buildingblock1_smiles'].values)
buildingblock2_smiles_vec = smile_vector(BRD4s['buildingblock2_smiles'].values)
buildingblock3_smiles_vec = smile_vector(BRD4s['buildingblock3_smiles'].values)
BRD4predictions = BRD4Model.predict([buildingblock1_smiles_vec, buildingblock2_smiles_vec, buildingblock3_smiles_vec, BRD4s[data_cols]])
BRD4s['binds'] = BRD4predictions.flatten()

17465/17465 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step


/tmp/ipykernel_24/1342692181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BRD4s['binds'] = BRD4predictions.flatten()


In [8]:
pd.concat([sEHs, BRD4s, HSAs])[['id', 'binds']].to_csv('submition.csv', index=False)